[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)

# Word Embedding (word2vec)

This notebook is a PyTorch adaptation of the [mxnet implementation of word2vec](http://www.d2l.ai/chapter_natural-language-processing-pretraining/word2vec.html) of the book [Dive into Deep Learning](http://www.d2l.ai/index.html). 

**Update (2023)** there is now a PyTorch implementation provided in [Pretraining word2vec](http://www.d2l.ai/chapter_natural-language-processing-pretraining/word2vec-pretraining.html#pretraining-word2vec) which is sligthly different form the one proposed here.

A natural language is a complex system that we use to express meanings. In this system, words are the basic unit of linguistic meaning. As its name implies, a word vector is a vector used to represent a word. It can also be thought of as the feature vector of a word. The technique of mapping words to vectors of real numbers is also known as word embedding. Over the last few years, word embedding has gradually become basic knowledge in natural language processing.

## Why not Use One-hot Vectors?


Although one-hot word vectors are easy to construct, they are usually not a good choice. One of the major reasons is that the one-hot word vectors cannot accurately express the similarity between different words, such as the cosine similarity that we commonly use. For the vectors $\boldsymbol{x}, \boldsymbol{y} \in \mathbb{R}^d$, their cosine similarities are the cosines of the angles between them:

$$\frac{\boldsymbol{x}^\top \boldsymbol{y}}{\|\boldsymbol{x}\| \|\boldsymbol{y}\|} \in [-1, 1].$$

Since the cosine similarity between the one-hot vectors of any two different words is 0, it is difficult to use the one-hot vector to accurately represent the similarity between multiple different words.

[Word2vec](https://code.google.com/archive/p/word2vec/) is a tool to solve the problem above.  It represents each word with a fixed-length vector and uses these vectors to better indicate the similarity and analogy relationships between different words. The Word2vec tool contains two models: [skip-gram](Distributed representations of words and phrases and their compositionality.) and continuous bag of words [CBOW](Efficient estimation of word representations in vector space). Next, we will take a look at the two models and their training methods.


## The Skip-Gram Model

The skip-gram model assumes that a word can be used to generate the words that surround it in a text sequence. For example, we assume that the text sequence is "the", "man", "loves", "his", and "son". We use "loves" as the central target word and set the context window size to 2. As shown below, given the central target word "loves", the skip-gram model is concerned with the conditional probability for generating the context words, "the", "man", "his" and "son", that are within a distance of no more than 2 words, which is

$$\mathbb{P}(\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}\mid\textrm{"loves"}).$$

We assume that, given the central target word, the context words are generated independently of each other. In this case, the formula above can be rewritten as

$$\mathbb{P}(\textrm{"the"}\mid\textrm{"loves"})\cdot\mathbb{P}(\textrm{"man"}\mid\textrm{"loves"})\cdot\mathbb{P}(\textrm{"his"}\mid\textrm{"loves"})\cdot\mathbb{P}(\textrm{"son"}\mid\textrm{"loves"}).$$

![The skip-gram model cares about the conditional probability of generating context words for a given central target word. ](https://www.di.ens.fr/~lelarge/skip-gram.svg)


In the skip-gram model, each word is represented as two $d$-dimension vectors, which are used to compute the conditional probability. We assume that the word is indexed as $i$ in the dictionary, its vector is represented as $\boldsymbol{v}_i\in\mathbb{R}^d$ when it is the central target word, and $\boldsymbol{u}_i\in\mathbb{R}^d$ when it is a context word.  Let the central target word $w_c$ and context word $w_o$ be indexed as $c$ and $o$ respectively in the dictionary. The conditional probability of generating the context word for the given central target word can be obtained by performing a softmax operation on the vector inner product:

$$\mathbb{P}(w_o \mid w_c) = \frac{\text{exp}(\boldsymbol{u}_o^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)},$$

where vocabulary index set $\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$. Assume that a text sequence of length $T$ is given, where the word at time step $t$ is denoted as $w^{(t)}$. Assume that context words are independently generated given center words. When context window size is $m$, the likelihood function of the skip-gram model is the joint probability of generating all the context words given any center word

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} \mathbb{P}(w^{(t+j)} \mid w^{(t)}),$$

After the training, for any word in the dictionary with index $i$, we are going to get its two word vector sets $\boldsymbol{v}_i$ and $\boldsymbol{u}_i$.  In applications of natural language processing (NLP), the central target word vector in the skip-gram model is generally used as the representation vector of a word.

## The Continuous Bag Of Words (CBOW) Model

The continuous bag of words (CBOW) model is similar to the skip-gram model. The biggest difference is that the CBOW model assumes that the central target word is generated based on the context words before and after it in the text sequence. With the same text sequence "the", "man", "loves", "his" and "son", in which "loves" is the central target word, given a context window size of 2, the CBOW model is concerned with the conditional probability of generating the target word "loves" based on the context words "the", "man", "his" and "son"(as shown below), such as

$$\mathbb{P}(\textrm{"loves"}\mid\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}).$$

![The CBOW model cares about the conditional probability of generating the central target word from given context words.  ](https://www.di.ens.fr/~lelarge/cbow.svg)

Since there are multiple context words in the CBOW model, we will average their word vectors and then use the same method as the skip-gram model to compute the conditional probability. We assume that $\boldsymbol{v_i}\in\mathbb{R}^d$ and $\boldsymbol{u_i}\in\mathbb{R}^d$ are the context word vector and central target word vector of the word with index $i$ in the dictionary (notice that the symbols are opposite to the ones in the skip-gram model). Let central target word $w_c$ be indexed as $c$, and context words $w_{o_1}, \ldots, w_{o_{2m}}$ be indexed as $o_1, \ldots, o_{2m}$ in the dictionary. Thus, the conditional probability of generating a central target word from the given context word is

$$\mathbb{P}(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\text{exp}\left(\frac{1}{2m}\boldsymbol{u}_c^\top (\boldsymbol{v}_{o_1} + \ldots + \boldsymbol{v}_{o_{2m}}) \right)}{ \sum_{i \in \mathcal{V}} \text{exp}\left(\frac{1}{2m}\boldsymbol{u}_i^\top (\boldsymbol{v}_{o_1} + \ldots + \boldsymbol{v}_{o_{2m}}) \right)}.$$


For brevity, denote $\mathcal{W}_o= \{w_{o_1}, \ldots, w_{o_{2m}}\}$, and $\bar{\boldsymbol{v}}_o = \left(\boldsymbol{v}_{o_1} + \ldots + \boldsymbol{v}_{o_{2m}} \right)/(2m)$. The equation above can be simplified as

$$\mathbb{P}(w_c \mid \mathcal{W}_o) = \frac{\exp\left(\boldsymbol{u}_c^\top \bar{\boldsymbol{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\boldsymbol{u}_i^\top \bar{\boldsymbol{v}}_o\right)}.$$

Given a text sequence of length $T$, we assume that the word at time step $t$ is $w^{(t)}$, and the context window size is $m$.  The likelihood function of the CBOW model is the probability of generating any central target word from the context words.

$$ \prod_{t=1}^{T}  \mathbb{P}(w^{(t)} \mid  w^{(t-m)}, \ldots,  w^{(t-1)},  w^{(t+1)}, \ldots,  w^{(t+m)}).$$

 Unlike the skip-gram model, we usually use the context word vector as the representation vector for a word in the CBOW model.

# Approximate Training

The core feature of the skip-gram model is the use of softmax operations to compute the conditional probability of generating context word $w_o$ based on the given central target word $w_c$.

$$\mathbb{P}(w_o \mid w_c) = \frac{\text{exp}(\boldsymbol{u}_o^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}.$$

The logarithmic loss corresponding to the conditional probability is given as

$$-\log \mathbb{P}(w_o \mid w_c) =
-\boldsymbol{u}_o^\top \boldsymbol{v}_c + \log\left(\sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)\right).$$


Because the softmax operation has considered that the context word could be any word in the dictionary $\mathcal{V}$, the loss mentioned above actually includes the sum of the number of items in the dictionary size. Hence, the gradient computation for each step contains the sum of the number of items in the dictionary size. For larger dictionaries with hundreds of thousands or even millions of words, the overhead for computing each gradient may be too high.  In order to reduce such computational complexity, we will introduce an approximate training method in this section: negative sampling. Since there is no major difference between the skip-gram model and the CBOW model, we will only use the skip-gram model as an example to introduce these two training methods in this section.



## Negative Sampling

Negative sampling modifies the original objective function. Given a context window for the central target word $w_c$, we will treat it as an event for context word $w_o$ to appear in the context window and compute the probability of this event from

$$\mathbb{P}(D=1\mid w_c, w_o) = \sigma(\boldsymbol{u}_o^\top \boldsymbol{v}_c),$$

Here, the $\sigma$ function has the same definition as the sigmoid activation function:

$$\sigma(x) = \frac{1}{1+\exp(-x)}.$$

We will first consider training the word vector by maximizing the joint probability of all events in the text sequence. Given a text sequence of length $T$, we assume that the word at time step $t$ is $w^{(t)}$ and the context window size is $m$. Now we consider maximizing the joint probability

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} \mathbb{P}(D=1\mid w^{(t)}, w^{(t+j)}).$$

However, the events included in the model only consider positive examples. In this case, only when all the word vectors are equal and their values approach infinity can the joint probability above be maximized to 1. Obviously, such word vectors are meaningless. Negative sampling makes the objective function more meaningful by sampling with an addition of negative examples. Assume that event $P$ occurs when context word $w_o$ to appear in the context window of central target word $w_c$, and we sample $K$ words that do not appear in the context window according to the distribution $\mathbb{P}(w)$ to act as noise words. We assume the event for noise word $w_k$($k=1, \ldots, K$) to not appear in the context window of central target word $w_c$ is $N_k$. Suppose that events $P and N_1, \ldots, N_K$ for both positive and negative examples are independent of each other. By considering negative sampling, we can rewrite the joint probability above, which only considers the positive examples, as


$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} \mathbb{P}(w^{(t+j)} \mid w^{(t)}),$$

Here, the conditional probability is approximated to be
$$ \mathbb{P}(w^{(t+j)} \mid w^{(t)}) =\mathbb{P}(D=1\mid w^{(t)}, w^{(t+j)})\prod_{k=1,\ w_k \sim \mathbb{P}(w)}^K \mathbb{P}(D=0\mid w^{(t)}, w_k).$$


Let the text sequence index of word $w^{(t)}$ at time step $t$ be $i_t$ and $h_k$ for noise word $w_k$ in the dictionary. The logarithmic loss for the conditional probability above is

$$
\begin{aligned}
-\log\mathbb{P}(w^{(t+j)} \mid w^{(t)})
=& -\log\mathbb{P}(D=1\mid w^{(t)}, w^{(t+j)}) - \sum_{k=1,\ w_k \sim \mathbb{P}(w)}^K \log\mathbb{P}(D=0\mid w^{(t)}, w_k)\\
=&-  \log\, \sigma\left(\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim \mathbb{P}(w)}^K \log\left(1-\sigma\left(\boldsymbol{u}_{h_k}^\top \boldsymbol{v}_{i_t}\right)\right)\\
=&-  \log\, \sigma\left(\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim \mathbb{P}(w)}^K \log\sigma\left(-\boldsymbol{u}_{h_k}^\top \boldsymbol{v}_{i_t}\right).
\end{aligned}
$$

Here, the gradient computation in each step of the training is no longer related to the dictionary size, but linearly related to $K$. When $K$ takes a smaller constant, the negative sampling has a lower computational overhead for each step.

For more details see [On word embeddings - Part 2: Approximating the Softmax](https://www.ruder.io/word-embeddings-softmax/)

# Implementation of Word2vec



In [1]:
import collections

import math
import numpy as np

import random
import sys
import time
import zipfile

## Process the Data Set

Penn Tree Bank (PTB) is a small commonly-used [corpus](https://github.com/tomsercu/lstm/tree/master/data). It takes samples from Wall Street Journal articles and includes training sets, validation sets, and test sets. We will train the word embedding model on the PTB training set. Each line of the data set acts as a sentence. All the words in a sentence are separated by spaces.

In [2]:
## colab SETUP
ROOT_DIR='/home/andy/00_workspace/dl/data/08/content'

# %cd $ROOT_DIR
# !mkdir data
# %cd data
# !wget https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt
# # ROOT_DIR='content'

import os
# from pathlib import Path

# ROOT_DIR = Path.home()

In [3]:
data_path = os.path.join(ROOT_DIR,'data/')
file = 'ptb.train.txt'
with open(data_path+file, 'r') as f:
    lines = f.readlines()
    # st is the abbreviation of "sentence" in the loop
    raw_dataset = [st.split() for st in lines]

'# sentences: %d' % len(raw_dataset)

'# sentences: 42068'

For the first three sentences of the data set, print the number of words and the first five words of each sentence. The end character of this data set is "&lt;eos&gt;", uncommon words are all represented by "&lt;unk&gt;", and numbers are replaced with "N".

In [4]:
sample_ds = [sen[:5] for sen in raw_dataset[:3]]
sample_ds


[['aer', 'banknote', 'berlitz', 'calloway', 'centrust'],
 ['pierre', '<unk>', 'N', 'years', 'old'],
 ['mr.', '<unk>', 'is', 'chairman', 'of']]

### Create Word Index

For the sake of simplicity, we only keep words that appear at least 5 times in the data set.

In [5]:
# tk is an abbreviation for "token" in the loop
# counter is cllections of the frequency of each token
counter = collections.Counter([tk for st in raw_dataset for tk in st])
counter = dict(filter(lambda x: x[1] >= 5, counter.items()))

In [6]:
idx_to_token = [tk for tk, _ in counter.items()]
token_to_idx = {tk: idx for idx, tk in enumerate(idx_to_token)}
dataset = [[token_to_idx[tk] for tk in st if tk in token_to_idx]
           for st in raw_dataset]
num_tokens = sum([len(st) for st in dataset])
'# tokens: %d' % num_tokens

'# tokens: 887100'

In [7]:
idx_to_token[:5]

['pierre', '<unk>', 'N', 'years', 'old']

In [8]:
token_to_idx['consensus']

4827

### Subsampling

In text data, there are generally some words that appear at high frequencies, such "the", "a", and "in" in English. Generally speaking, in a context window, it is better to train the word embedding model when a word (such as "chip") and a lower-frequency word (such as "microprocessor") appear at the same time, rather than when a word appears with a higher-frequency word (such as "the"). Therefore, when training the word embedding model, we can perform subsampling[2] on the words. Specifically, each indexed word $w_i$ in the data set will drop out at a certain probability. The dropout probability is given as:

$$ \mathbb{P}(w_i) = \max\left(1 - \sqrt{\frac{t}{f(w_i)}}, 0\right),$$

Here, $f(w_i)$ is the ratio of the instances of word $w_i$ to the total number of words in the data set, and the constant $t$ is a hyper-parameter (set to $10^{-4}$ in this experiment). As we can see, it is only possible to drop out the word $w_i$ in subsampling when $f(w_i) > t$. The higher the word's frequency, the higher its dropout probability.

In [9]:
def discard(idx):
    return random.uniform(0, 1) < 1 - math.sqrt(
        1e-4 / counter[idx_to_token[idx]] * num_tokens)

subsampled_dataset = [[tk for tk in st if not discard(tk)] for st in dataset]
'# tokens: %d' % sum([len(st) for st in subsampled_dataset])

'# tokens: 375552'

In [10]:
def compare_counts(token):
    return '# %s: before=%d, after=%d' % (token, sum(
        [st.count(token_to_idx[token]) for st in dataset]), sum(
        [st.count(token_to_idx[token]) for st in subsampled_dataset]))

compare_counts('the')

'# the: before=50770, after=2085'

In [11]:
compare_counts('join')

'# join: before=45, after=45'

### Extract Central Target Words and Context Words

We use words with a distance from the central target word not exceeding the context window size as the context words of the given center target word. The following definition function extracts all the central target words and their context words. It uniformly and randomly samples an integer to be used as the context window size between integer 1 and the `max_window_size` (maximum context window).

In [12]:
def get_centers_and_contexts(dataset, max_window_size):
    centers, contexts = [], []
    for st in dataset:
        # Each sentence needs at least 2 words to form a
        # "central target word - context word" pair
        if len(st) < 2:
            continue
        centers += st
        for center_i in range(len(st)):
            window_size = random.randint(1, max_window_size)
            indices = list(range(max(0, center_i - window_size),
                                 min(len(st), center_i + 1 + window_size)))
            # Exclude the central target word from the context words
            indices.remove(center_i)
            contexts.append([st[idx] for idx in indices])
    return centers, contexts

In [13]:
tiny_dataset = [list(range(7)), list(range(7, 10))]
print('dataset', tiny_dataset)
for center, context in zip(*get_centers_and_contexts(tiny_dataset, 2)):
    print('center', center, 'has contexts', context)

dataset [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
center 0 has contexts [1]
center 1 has contexts [0, 2]
center 2 has contexts [0, 1, 3, 4]
center 3 has contexts [2, 4]
center 4 has contexts [3, 5]
center 5 has contexts [3, 4, 6]
center 6 has contexts [5]
center 7 has contexts [8]
center 8 has contexts [7, 9]
center 9 has contexts [7, 8]


In the experiment, we set the maximum context window size to 5. The following extracts all the central target words and their context words in the data set.

In [14]:
all_centers, all_contexts = get_centers_and_contexts(subsampled_dataset, 5)

## Negative Sampling

We use negative sampling for approximate training. For a central and context word pair, we randomly sample $K$ noise words ($K=5$ in the experiment). According to the suggestion in the Word2vec paper, the noise word sampling probability $\mathbb{P}(w)$ is the ratio of the word frequency of $w$ to the total word frequency raised to the power of 0.75.

In [15]:
def get_negatives(all_contexts, sampling_weights, K):
    all_negatives, neg_candidates, i = [], [], 0
    population = list(range(len(sampling_weights)))
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            if i == len(neg_candidates):
                # An index of k words is randomly generated as noise words
                # based on the weight of each word (sampling_weights). For
                # efficient calculation, k can be set slightly larger
                i, neg_candidates = 0, random.choices(population, sampling_weights, k=int(1e5))
            neg, i = neg_candidates[i], i + 1
            # Noise words cannot be context words
            if neg not in set(contexts):
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives

sampling_weights = [counter[w]**0.75 for w in idx_to_token]
all_negatives = get_negatives(all_contexts, sampling_weights, 5)

In [16]:
all_negatives[0]

[2078,
 348,
 828,
 1877,
 535,
 5777,
 453,
 5566,
 9466,
 10,
 9,
 8293,
 3853,
 3696,
 1285,
 2,
 39,
 636,
 906,
 44]

In [17]:
all_contexts[0]

[5, 6, 11, 12]

## Reading Data

We extracted all central target words `all_centers`, and the context words `all_contexts` and noise words `all_negatives` of each central target word from the data set. We will read them in random mini-batches.

In a mini-batch of data, the $i$-th example includes a central word and its corresponding $n_i$ context words and $m_i$ noise words. Since the context window size of each example may be different, the sum of context words and noise words, $n_i+m_i$, will be different. When constructing a mini-batch, we concatenate the context words and noise words of each example, and add 0s for padding until the length of the concatenations are the same, that is, the length of all concatenations is $\max_i n_i+m_i$(`max_len`). In order to avoid the effect of padding on the loss function calculation, we construct the mask variable `masks`, each element of which corresponds to an element in the concatenation of context and noise words, `contexts_negatives`. When an element in the variable `contexts_negatives` is a padding, the element in the mask variable `masks` at the same position will be 0. Otherwise, it takes the value 1. In order to distinguish between positive and negative examples, we also need to distinguish the context words from the noise words in the `contexts_negatives` variable. Based on the construction of the mask variable, we only need to create a label variable `labels` with the same shape as the `contexts_negatives` variable and set the elements corresponding to context words (positive examples) to 1, and the rest to 0.

Next, we will implement the mini-batch reading function `batchify`. Its mini-batch input `data` is a list whose length is the batch size, each element of which contains central target words `center`, context words `context`, and noise words `negative`. The mini-batch data returned by this function conforms to the format we need, for example, it includes the mask variable. We wrap this function in a `Dataset` module.

In [18]:
import torch
import torch.nn as nn

In [19]:
from torch.utils.data import Dataset, DataLoader

In [20]:
class PTB_dataset(Dataset):
    
    def __init__(self, all_centers, all_contexts, all_negatives):
        self.all_centers, self.all_contexts_negatives, self.all_masks, self.all_labels = self.batchify(list(zip(all_centers,all_contexts,all_negatives)))
        
    def __len__(self):
        return len(self.all_centers)
    
    def __getitem__(self,idx):
        return self.all_centers[idx], self.all_contexts_negatives[idx], self.all_masks[idx], self.all_labels[idx]
        
    def batchify(self,data):
        max_len = max(len(c) + len(n) for _, c, n in data)
        centers, contexts_negatives, masks, labels = [], [], [], []
        for center, context, negative in data:
            cur_len = len(context) + len(negative)
            # list + list = concatenation
            centers += [center]
            contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
            masks += [[1] * cur_len + [0] * (max_len - cur_len)]
            labels += [[1] * len(context) + [0] * (max_len - len(context))]
        return (torch.tensor(centers).view((-1, 1)), torch.tensor(np.array(contexts_negatives)),
            torch.tensor(np.array(masks)), torch.tensor(np.array(labels)))
        

In [21]:
ptbdata = PTB_dataset(all_centers, all_contexts, all_negatives)

In [22]:
ptbdata[1]

(tensor([5]),
 tensor([   0,    6,   11,   12,   13,  451,  135, 7703,  161, 1348,  102, 4532,
          912,  623, 4204,    9, 1765, 2214,   76, 6821, 2975,  685, 2922,  219,
         4571, 1729, 4669, 2233, 1463, 1290,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

And here is our dataloader.

In [23]:
batch_size = 512

data_loader = DataLoader(ptbdata, batch_size, shuffle=True,
                              num_workers=4)
for batch in data_loader:
    for name, data in zip(['centers', 'contexts_negatives', 'masks',
                           'labels'], batch):
        print(name, 'shape:', data.shape)
    break

centers shape: torch.Size([512, 1])
contexts_negatives shape: torch.Size([512, 60])
masks shape: torch.Size([512, 60])
labels shape: torch.Size([512, 60])


## The Skip-Gram Model

You will implement the skip-gram model by using embedding layers and mini-batch multiplication [`torch.einsum`](https://pytorch.org/docs/stable/torch.html#torch.bmm). These methods are also often used to implement other natural language processing applications.

In [24]:
# taken from the spotlight library, see
# https://github.com/maciejkula/spotlight/blob/master/spotlight/layers.py
class ScaledEmbedding(nn.Embedding):
    """
    Embedding layer that initialises its values
    to using a normal variable scaled by the inverse
    of the emedding dimension.
    """
    def reset_parameters(self):
        """
        Initialize parameters.
        """
        self.weight.data.normal_(0, 1.0 / self.embedding_dim)
        if self.padding_idx is not None:
            self.weight.data[self.padding_idx].fill_(0)

### Skip-gram Model Forward Calculation

In forward calculation, the input of the skip-gram model contains the central target word index `center` and the concatenated context and noise word index `contexts_and_negatives`. In which, the `center` variable has the shape (batch size, 1), while the `contexts_and_negatives` variable has the shape (batch size, `max_len`). These two variables are first transformed from word indexes to word vectors by the word embedding layer, and then the output of shape (batch size, 1, `max_len`) is obtained by mini-batch multiplication. Each element in the output is the inner product of the central target word vector and the context word vector or noise word vector.

In [25]:
class Skip_gram(nn.Module):
    def __init__(self, input_dim, embed_size = 100):
        super(Skip_gram, self).__init__()
        self.input_dim = input_dim
        self.embed_size = embed_size
        self.central_emb = ScaledEmbedding(self.input_dim,self.embed_size)
        self.context_emb = ScaledEmbedding(self.input_dim,self.embed_size)
        
    def forward(self, icent, icont):
        #
        # your code here (hint: dimensions are for icent (bs,1) for icont (bs,max_len) and output (bs,1,max_len))
        #
        cent = self.central_emb(icent)  # (bs, 1, embed_size)
        cont = self.context_emb(icont)  # (bs, max_len, embed_size)
        return torch.bmm(cent, cont.permute(0, 2, 1))  # (bs, 1, max_len)
        

In [26]:
net = Skip_gram(len(idx_to_token))
len(idx_to_token)

9858

In [27]:
out = net(torch.tensor([0,1,3]).unsqueeze(1),torch.tensor([[0,0],[0,0],[0,0]]))
out, out.shape

(tensor([[[ 0.0005,  0.0005]],
 
         [[ 0.0009,  0.0009]],
 
         [[-0.0014, -0.0014]]], grad_fn=<BmmBackward0>),
 torch.Size([3, 1, 2]))

### Loss function

It is worth mentioning that we need use the mask variable to specify the partial predicted value and label that participate in loss function calculation in the mini-batch: when the mask is 1, the predicted value and label of the corresponding position will participate in the calculation of the loss function; When the mask is 0, the predicted value and label of the corresponding position do not participate in the calculation of the loss function. As we mentioned earlier, mask variables can be used to avoid the effect of padding on loss function calculations.

In [28]:
loss_fn = nn.BCEWithLogitsLoss(reduction='none')
def criterion(pred, label, mask):
    #
    # your code here
    #
    # pred_flatten, label_flatten, mask_flatten = pred.view(-1), label.view(-1), mask.view(-1)
    # pred_mask = pred.mul(label*2-1)
    # dot_val = pred.mul(label*2-1)
    # smd_val = torch.sigmoid(dot_val)
    # log_smd = torch.log(smd_val)
    # print(f"dot_val={dot_val}, smd_val={smd_val}, log_smd={log_smd}")
    # pred_logsig = -torch.nn.functional.logsigmoid(pred.mul(label*2-1))
    # print(f"pred_logsig={pred_logsig}")
    # pos_logsig = -torch.log(torch.masked_select(pred_sig, label))
    # neg_logsig = -torch.log(1 - torch.masked_select(torch.masked_select(pred_sig, mask), 1 - label))
    # print(f"pred_logsig.shape={pred_logsig.shape}")
    # pos_logsig_sum = pred_logsig.mul(label).sum(dim=1)
    # neg_logsig_sum = pred_logsig.mul(mask).mul(torch.ones_like(label)-label).sum(dim=1)
    # print(f"pred_logsig.shape={pred_logsig.shape}, mask.shape={mask.shape}, pred.shape={pred.shape}, label.shape={label.shape}")
    # logsim_mean = pred_logsig.mul(mask).sum(dim=1)/mask.sum(dim=1)
    # pos_len = label.sum(dim=1).unsqueeze(1)
    # mask_len = mask.sum(dim=1)
    # print(f'pos.shape={pos_logsig_sum.shape}, neg.shape={neg_logsig_sum.shape}, label_len.shape={lable_len.shape}')
    # return (pos_logsig_sum + neg_logsig_sum.mul(pos_len)).mean()
    # print(f'pos_logsig_sum={pos_logsig_sum}, neg_logsig_sum={neg_logsig_sum}, lable_len={lable_len}')
    # print(f"logsim_mean={logsim_mean}")
    # return torch.exp(logsim_mean).mean()
    # return logsim_mean.mean()
    # print(f"pred={pred[0:5,0:5]}")
    # print(f"label={label[0:5,0:5]}")
    return (loss_fn(pred, label).mul(mask).sum(dim=1)/mask.sum(dim=1)).mean()

In [29]:
pred = torch.tensor([[1.5, 0.3, -1, 2], [1.1, -0.6, 2.2, 0.4]])
# 1 and 0 in the label variables label represent context words and the noise
# words, respectively
label = torch.tensor([[1, 0, 0, 0], [1, 1, 0, 0]]).type(torch.FloatTensor)
mask = torch.tensor([[1, 1, 1, 1], [1, 1, 1, 0]]).type(torch.FloatTensor)  # Mask variable

criterion(pred,label,mask)

tensor(1.0420)

In [30]:
optimizer = torch.optim.Adam(net.parameters(),lr=0.005)

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

In [32]:
def train(n_epochs):
    net.train()
    for epoch in range(n_epochs):
        start, loss = time.time(), 0
        b_cnt = 0
        for cent, cont, mask, label in data_loader:
            #
            # your code here
            #
            output = net(cent.to(device), cont.to(device))
            cur_loss = criterion(output.squeeze(1), label.to(device).float(), mask.to(device))
            
            optimizer.zero_grad()
            cur_loss.backward()
            optimizer.step()
            
            loss += cur_loss.detach().cpu().item()
            b_cnt += 1
            # return
            
        loss /= b_cnt
        print('epoch %d, loss %.4f, time %.2fs'
              % (epoch + 1, loss, time.time() - start))

In [34]:
train(100)

epoch 1, loss 0.2253, time 3.37s
epoch 2, loss 0.2251, time 3.30s
epoch 3, loss 0.2252, time 3.43s
epoch 4, loss 0.2250, time 3.49s
epoch 5, loss 0.2250, time 3.42s
epoch 6, loss 0.2250, time 3.32s
epoch 7, loss 0.2248, time 3.19s
epoch 8, loss 0.2248, time 3.15s
epoch 9, loss 0.2247, time 3.19s
epoch 10, loss 0.2246, time 3.20s
epoch 11, loss 0.2246, time 3.15s
epoch 12, loss 0.2246, time 3.16s
epoch 13, loss 0.2246, time 3.14s
epoch 14, loss 0.2245, time 3.20s
epoch 15, loss 0.2244, time 3.19s
epoch 16, loss 0.2244, time 3.19s
epoch 17, loss 0.2243, time 3.20s
epoch 18, loss 0.2243, time 3.18s
epoch 19, loss 0.2241, time 3.22s
epoch 20, loss 0.2242, time 3.20s
epoch 21, loss 0.2241, time 3.14s
epoch 22, loss 0.2241, time 3.17s
epoch 23, loss 0.2240, time 3.24s
epoch 24, loss 0.2240, time 3.26s
epoch 25, loss 0.2239, time 3.22s
epoch 26, loss 0.2238, time 3.30s
epoch 27, loss 0.2238, time 3.33s
epoch 28, loss 0.2238, time 3.21s
epoch 29, loss 0.2237, time 3.26s
epoch 30, loss 0.2238, 

## Applying the Word Embedding Model

After training the word embedding model, we can represent similarity in meaning between words based on the cosine similarity of two word vectors. As we can see, when using the trained word embedding model, the words closest in meaning to the word "chip" are mostly related to chips.

In [35]:
def get_similar_tokens(query_token, k, W):
    #
    # your code here
    #
    cos = torch.cosine_similarity(W, W[token_to_idx[query_token]].unsqueeze(0), dim=1)
    _,topk = torch.topk(cos, k=k+1,)
    for i in topk[1:]:# Remove the input words
        print('[%s]cosine sim=%.3f: %s' % (query_token, cos[i], (idx_to_token[i])))

get_similar_tokens('chip', 3, net.central_emb.weight.data)
get_similar_tokens('book', 3, net.central_emb.weight.data)

[chip]cosine sim=0.469: models
[chip]cosine sim=0.403: pinpoint
[chip]cosine sim=0.400: builds
[book]cosine sim=0.428: categories
[book]cosine sim=0.396: shadow
[book]cosine sim=0.372: variations


In [148]:
# test
x1 = torch.tensor([[1.,2.,3.]])
x2 = torch.tensor([[4.,5.,6.], [7.,8.,9.],[2.,4.,6.]])
cos = torch.cosine_similarity(x1, x2, dim=1)
cos

tensor([0.9746, 0.9594, 1.0000])

[![Dataflowr](https://raw.githubusercontent.com/dataflowr/website/master/_assets/dataflowr_logo.png)](https://dataflowr.github.io/website/)